In [14]:
import pandas as pd
import numpy as np

import scipy.sparse as sparse

from implicit.als import AlternatingLeastSquares

In [15]:
users_tags = pd.read_csv('/Users/max/Downloads/hetrec2011-delicious-2k/user_taggedbookmarks-timestamps.dat', delimiter='\t', header=None, 
        names=['user_id', 'bookmark_id', 'tag_id', 'timestamp'], engine='python')

In [16]:
tags = pd.read_csv('/Users/max/Downloads/hetrec2011-delicious-2k/bookmark_tags.dat', delimiter='\t', header=None, 
        names=['bookmark_id', 'tag_id', 'tagWeight'], engine='python')

In [17]:
data = pd.merge(users_tags, tags,  how='left', left_on=['bookmark_id','tag_id'], right_on = ['bookmark_id','tag_id'])

In [18]:
data.head(2)

,user_id,bookmark_id,tag_id,timestamp,tagWeight
0,userID,bookmarkID,tagID,timestamp,tagWeight
1,8,1,1,1289255362000,NaN


In [19]:
data = data.drop(0)

In [20]:
data.reset_index(inplace=True)

In [21]:
data.head()

,index,user_id,bookmark_id,tag_id,timestamp,tagWeight
0,1,8,1,1,1289255362000,NaN
1,2,8,2,1,1289255159000,4
2,3,8,7,1,1289238901000,NaN
3,4,8,7,6,1289238901000,NaN
4,5,8,7,7,1289238901000,NaN


In [22]:
data['tagWeight'] = data['tagWeight'].fillna(1)

In [23]:
data['tag_id'].isna().unique()

array([False])

In [24]:
data = data[['user_id','bookmark_id','tagWeight']]

In [25]:
data['user_id'] = data['user_id'].astype("category").cat.codes
data['bookmark_id'] = data['bookmark_id'].astype("category").cat.codes

In [26]:
rows = data.user_id.astype(int)
cols = data.bookmark_id.astype(int)
tagWeight = data.tagWeight.astype(int)

In [27]:
users = list(data.user_id.unique())
bookmarks = list(data.bookmark_id.unique())

In [28]:
data_sparse = sparse.csr_matrix((tagWeight, (cols, rows)), shape=(len(bookmarks),len(users)))

In [29]:
data_sparse = sparse.csr_matrix((tagWeight, (cols, rows)), shape=(len(bookmarks),len(users)))

In [31]:
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

100%|██████████| 15.0/15 [00:03<00:00,  4.89it/s]


In [34]:
userid = 100

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [35]:
recommendations

[(20319, 1.2683402),
 (43089, 1.0611391),
 (43431, 0.93255067),
 (13055, 0.926544),
 (24663, 0.8419108),
 (9811, 0.798202),
 (5496, 0.78334117),
 (12983, 0.7812536),
 (10207, 0.77846587),
 (24189, 0.7704406)]